# Creating a climate index API

In order to know which destinations have (or will have) the best weather on a given date we will be using historical data from all of Spain's weather stations, as well as packaging that information into an API that will allow us to remotely perform queries.

Since all information from 291 weather stations are stored in individual *csv* files, let's import the first one and see how can we coax the necessary data out of it.

# Transforming historical data into a weather index

*Indices are particularly valuable because they allow the integrated effects of a range of climatic variables to be quantified, facilitating an interpretation and rating of climatic conditions at a destination. Another advantage of indices is that they enable the climate of tourism destinations to be objectively compared and are therefore a convenient and more conceptually sound means to assess possible impacts of climate change on the distribution of climatic resources worldwide.*

                  - An Inter-Comparison of the Holiday Climate Index (HCI) and the Tourism Climate Index (TCI) in Europe
                    https://www.mdpi.com/2073-4433/7/6/80/htm
                    
                    
                    
                    
Our aim is to condense all meteorological data (wind, temperature, hours of daylight, rain...) into a single numnber (an index) that can be used to easily compare one destination with another. The objective is to create a dataframe where every row is a different destination and each column holds the index for each week of the year.

We can't use the **TCI** as-is because we lack some of the data (humidity and cloud cover %), so we will have to make an index of our own. From now on we will call it simply the **CWI** (**C**ycling **W**eather **I**ndex).

## Creating our **CWI**

While it would be nice to have all weather data for every town in Spain, we have to make do with the available datasets.

Of the present variables in the official meteo datasets, the following variables are of use:


- Date (to group values per week).
- Average temperature.
- Rain (mm).
- Average wind speed.


The hours of sunlight aren't really meaningful because we're crafting this custom index to be used only on Spain, which has a single timezone and the latitude change isn't that great.


The original **TCI** roughly assigns the following weights to each parameter:


**Maximum temperature:** 40%

**Cloud cover:** 20%

**Precipitation:** 30%

**Wind:** 10%



While those percentages are quite good, this model was made with tourism in mind. Cycling has a few diferrences with tourism and other leisure activities that must be kept in mind when creating our **CWS**:

- Clouds are OK as long as it doesn't rain.
- Wind is very harmful.


A first approach to our index could be the following:

**Maximum temperature:** 40%

**Precipitation:** 40%

**Wind:** 20%

## Importing our data

The first step is importing our data. Every weather station's data is contained in a different *csv* file, we'll import the first one to see how data is structured.

In [38]:
import pandas as pd
import time
from pathlib import Path

In [39]:
#Importing our dataframe.

df = pd.read_csv('meteo_test.csv', sep=';')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17390 entries, 0 to 17389
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FECHA          17390 non-null  object 
 1   INDICATIVO     17390 non-null  object 
 2   NOMBRE         17390 non-null  object 
 3   PROVINCIA      17390 non-null  object 
 4   ALTITUD        17390 non-null  int64  
 5   TMEDIA         16212 non-null  float64
 6   PRECIPITACION  16081 non-null  object 
 7   TMIN           16212 non-null  float64
 8   HORATMIN       16191 non-null  object 
 9   TMAX           16231 non-null  float64
 10  HORATMAX       16218 non-null  object 
 11  DIR            15588 non-null  float64
 12  VELMEDIA       16233 non-null  float64
 13  RACHA          15569 non-null  float64
 14  HORARACHA      15539 non-null  object 
 15  SOL            10102 non-null  float64
 16  PRESMAX        8928 non-null   float64
 17  HORAPRESMAX    8899 non-null   object 
 18  PRESMI

In [41]:
df.head()

,FECHA,INDICATIVO,NOMBRE,PROVINCIA,ALTITUD,TMEDIA,PRECIPITACION,TMIN,HORATMIN,TMAX,HORATMAX,DIR,VELMEDIA,RACHA,HORARACHA,SOL,PRESMAX,HORAPRESMAX,PRESMIN,HORAPRESMIN
0,1968-03-01,0002I,VANDELL�S,TARRAGONA,32,8.9,21.0,6.6,03:00,11.2,18:00,5.0,1.9,6.7,10:55,0.0,NaN,NaN,NaN,NaN
1,1968-03-02,0002I,VANDELL�S,TARRAGONA,32,10.9,0.0,6.0,07:10,15.8,15:00,32.0,1.1,18.6,23:40,8.6,NaN,NaN,NaN,NaN
2,1968-03-03,0002I,VANDELL�S,TARRAGONA,32,NaN,0.0,NaN,NaN,NaN,NaN,32.0,6.1,19.2,00:55,8.6,NaN,NaN,NaN,NaN
3,1968-03-04,0002I,VANDELL�S,TARRAGONA,32,10.9,6.5,7.8,05:00,14.0,11:40,32.0,3.9,8.1,06:10,8.4,NaN,NaN,NaN,NaN
4,1968-03-05,0002I,VANDELL�S,TARRAGONA,32,NaN,0.0,NaN,NaN,NaN,NaN,32.0,3.6,13.1,23:40,2.5,NaN,NaN,NaN,NaN


In [42]:
#Precipitation needs to be changed to float, but first some values ('ip') will have to go (it means 0 precipitation).

df['PRECIPITACION'] = df['PRECIPITACION'].str.replace('Ip', '0.0')

df["PRECIPITACION"] = pd.to_numeric(df["PRECIPITACION"])

## Using time series to get the average values

Since we want to calculate the **TCI** on a weekly basis it makes sense to compact our data with the same granularity. For this purpose we will be using time series.

In [43]:
#Converting 'FECHA' to Datetime format.

df["FECHA"] = df['FECHA'].apply(pd.to_datetime)

In [44]:
#Checking the result.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17390 entries, 0 to 17389
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   FECHA          17390 non-null  datetime64[ns]
 1   INDICATIVO     17390 non-null  object        
 2   NOMBRE         17390 non-null  object        
 3   PROVINCIA      17390 non-null  object        
 4   ALTITUD        17390 non-null  int64         
 5   TMEDIA         16212 non-null  float64       
 6   PRECIPITACION  16081 non-null  float64       
 7   TMIN           16212 non-null  float64       
 8   HORATMIN       16191 non-null  object        
 9   TMAX           16231 non-null  float64       
 10  HORATMAX       16218 non-null  object        
 11  DIR            15588 non-null  float64       
 12  VELMEDIA       16233 non-null  float64       
 13  RACHA          15569 non-null  float64       
 14  HORARACHA      15539 non-null  object        
 15  SOL            1010

Since the climate is changing it would be foolish to use all data for our forecast. The last 5 years will suffice.

In [45]:
#Creating a dataframe with data from the last 5 years.

filtered = df.loc[(df['FECHA'] >= '2016-01-01') & (df['FECHA'] < '2021-01-01')]

#Reseting the index.

filtered.reset_index(drop=True, inplace=True)

In [46]:
filtered.head()

,FECHA,INDICATIVO,NOMBRE,PROVINCIA,ALTITUD,TMEDIA,PRECIPITACION,TMIN,HORATMIN,TMAX,HORATMAX,DIR,VELMEDIA,RACHA,HORARACHA,SOL,PRESMAX,HORAPRESMAX,PRESMIN,HORAPRESMIN
0,2016-01-01,0002I,VANDELL�S,TARRAGONA,32,13.6,0.1,10.8,04:40,16.4,13:00,12.0,0.6,3.6,00:10,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,0002I,VANDELL�S,TARRAGONA,32,13.6,0.0,10.7,21:10,16.4,14:20,99.0,2.8,13.1,10:30,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,0002I,VANDELL�S,TARRAGONA,32,12.1,0.0,9.6,23:00,14.6,11:50,24.0,1.4,8.6,09:30,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,0002I,VANDELL�S,TARRAGONA,32,12.7,0.3,9.4,01:00,16.0,04:40,33.0,1.7,14.2,20:40,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,0002I,VANDELL�S,TARRAGONA,32,12.0,0.0,9.7,23:59,14.4,13:30,30.0,3.1,17.5,03:10,NaN,NaN,NaN,NaN,NaN


In [47]:
#We don't have data for every single day, but this will be enough.

filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   FECHA          1291 non-null   datetime64[ns]
 1   INDICATIVO     1291 non-null   object        
 2   NOMBRE         1291 non-null   object        
 3   PROVINCIA      1291 non-null   object        
 4   ALTITUD        1291 non-null   int64         
 5   TMEDIA         1276 non-null   float64       
 6   PRECIPITACION  1126 non-null   float64       
 7   TMIN           1276 non-null   float64       
 8   HORATMIN       1274 non-null   object        
 9   TMAX           1276 non-null   float64       
 10  HORATMAX       1274 non-null   object        
 11  DIR            1236 non-null   float64       
 12  VELMEDIA       1241 non-null   float64       
 13  RACHA          1236 non-null   float64       
 14  HORARACHA      1236 non-null   object        
 15  SOL            0 non-

Now that we have a dataset of the last 5 years it's time to calculate the weekly averages.

In [48]:
#Using datetime to add the week number as a new column.

filtered['week'] = None
weekNumber = filtered['FECHA'].dt.week.tolist()
filtered['week'] = weekNumber

<ipython-input-48-39abab0bf700>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['week'] = None
<ipython-input-48-39abab0bf700>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  weekNumber = filtered['FECHA'].dt.week.tolist()
<ipython-input-48-39abab0bf700>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['week'] = weekNumber


In [49]:
filtered.head()

,FECHA,INDICATIVO,NOMBRE,PROVINCIA,ALTITUD,TMEDIA,PRECIPITACION,TMIN,HORATMIN,TMAX,...,DIR,VELMEDIA,RACHA,HORARACHA,SOL,PRESMAX,HORAPRESMAX,PRESMIN,HORAPRESMIN,week
0,2016-01-01,0002I,VANDELL�S,TARRAGONA,32,13.6,0.1,10.8,04:40,16.4,...,12.0,0.6,3.6,00:10,NaN,NaN,NaN,NaN,NaN,53
1,2016-01-02,0002I,VANDELL�S,TARRAGONA,32,13.6,0.0,10.7,21:10,16.4,...,99.0,2.8,13.1,10:30,NaN,NaN,NaN,NaN,NaN,53
2,2016-01-03,0002I,VANDELL�S,TARRAGONA,32,12.1,0.0,9.6,23:00,14.6,...,24.0,1.4,8.6,09:30,NaN,NaN,NaN,NaN,NaN,53
3,2016-01-04,0002I,VANDELL�S,TARRAGONA,32,12.7,0.3,9.4,01:00,16.0,...,33.0,1.7,14.2,20:40,NaN,NaN,NaN,NaN,NaN,1
4,2016-01-05,0002I,VANDELL�S,TARRAGONA,32,12.0,0.0,9.7,23:59,14.4,...,30.0,3.1,17.5,03:10,NaN,NaN,NaN,NaN,NaN,1


In [50]:
#Creating a dataframe with the weekly mean values.

df_mean = filtered.groupby("week").mean()

## Assigning each value a rating 0-10

To create our index all values (temperature, rain and wind speed) will need to be rated on a given scale, from 0 to 10. For this we will be using the same ratings as the famous **TCI**.

In [51]:
#Assigning temperature values.

df_mean['tmax'] = None

for i in range(len(df_mean)):
    if df_mean['TMAX'].iloc[i] >= 39: #Checking the temperature, from high to low.
        df_mean['tmax'].iloc[i] = 0 #Assigning the score.
    elif df_mean['TMAX'].iloc[i] > 38:
        df_mean['tmax'].iloc[i] = 1
    elif df_mean['TMAX'].iloc[i] > 37:
        df_mean['tmax'].iloc[i] = 2
    elif df_mean['TMAX'].iloc[i] > 36:
        df_mean['tmax'].iloc[i] = 3
    elif df_mean['TMAX'].iloc[i] > 35:
        df_mean['tmax'].iloc[i] = 4
    elif df_mean['TMAX'].iloc[i] > 33:
        df_mean['tmax'].iloc[i] = 5
    elif df_mean['TMAX'].iloc[i] > 31:
        df_mean['tmax'].iloc[i] = 6
    elif df_mean['TMAX'].iloc[i] > 29:
        df_mean['tmax'].iloc[i] = 7
    elif df_mean['TMAX'].iloc[i] > 27:
        df_mean['tmax'].iloc[i] = 8
    elif df_mean['TMAX'].iloc[i] > 26:
        df_mean['tmax'].iloc[i] = 9
    elif df_mean['TMAX'].iloc[i] > 23:
        df_mean['tmax'].iloc[i] = 10
    elif df_mean['TMAX'].iloc[i] > 20:
        df_mean['tmax'].iloc[i] = 9
    elif df_mean['TMAX'].iloc[i] > 19:
        df_mean['tmax'].iloc[i] = 8
    elif df_mean['TMAX'].iloc[i] > 18:
        df_mean['tmax'].iloc[i] = 7
    elif df_mean['TMAX'].iloc[i] > 15:
        df_mean['tmax'].iloc[i] = 6
    elif df_mean['TMAX'].iloc[i] > 11:
        df_mean['tmax'].iloc[i] = 5
    elif df_mean['TMAX'].iloc[i] > 7:
        df_mean['tmax'].iloc[i] = 4
    elif df_mean['TMAX'].iloc[i] > 0:
        df_mean['tmax'].iloc[i] > 3
    elif df_mean['TMAX'].iloc[i] <= 3:
        df_mean['tmax'].iloc[i] = 0

C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [52]:
#Assigning precipitation values.

df_mean['rain'] = None

for i in range(len(df_mean)):
    if df_mean['PRECIPITACION'].iloc[i] >= 40:
        df_mean['rain'].iloc[i] = -2
    elif df_mean['PRECIPITACION'].iloc[i] >= 25:
        df_mean['rain'].iloc[i] = -1
    elif df_mean['PRECIPITACION'].iloc[i] > 12:
        df_mean['rain'].iloc[i] = 0
    elif df_mean['PRECIPITACION'].iloc[i] > 10:
        df_mean['rain'].iloc[i] = 1
    elif df_mean['PRECIPITACION'].iloc[i] > 9:
        df_mean['rain'].iloc[i] = 2
    elif df_mean['PRECIPITACION'].iloc[i] > 8:
        df_mean['rain'].iloc[i] = 3
    elif df_mean['PRECIPITACION'].iloc[i] > 7:
        df_mean['rain'].iloc[i] = 4
    elif df_mean['PRECIPITACION'].iloc[i] > 6:
        df_mean['rain'].iloc[i] = 5
    elif df_mean['PRECIPITACION'].iloc[i] > 5:
        df_mean['rain'].iloc[i] = 6
    elif df_mean['PRECIPITACION'].iloc[i] > 4:
        df_mean['rain'].iloc[i] = 7
    elif df_mean['PRECIPITACION'].iloc[i] > 3:
        df_mean['rain'].iloc[i] = 8
    elif df_mean['PRECIPITACION'].iloc[i] > 0:
        df_mean['rain'].iloc[i] = 9
    elif df_mean['PRECIPITACION'].iloc[i] == 0.0:
        df_mean['rain'].iloc[i] = 10

C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [53]:
#Assigning wind values. Since our values are in m/s and we need Km/h, a conversion is needed.

df_mean['wind'] = None

for i in range(len(df_mean)):
    if df_mean['VELMEDIA'].iloc[i]*3.6 > 70:
        df_mean['wind'].iloc[i] = -10
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 65:
        df_mean['wind'].iloc[i] = -8
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 60:
        df_mean['wind'].iloc[i] = -6
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 55:
        df_mean['wind'].iloc[i] = -4
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 50:
        df_mean['wind'].iloc[i] = -2
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 40:
        df_mean['wind'].iloc[i] = 0
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 35:
        df_mean['wind'].iloc[i] = 2
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 30:
        df_mean['wind'].iloc[i] = 6
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 25:
        df_mean['wind'].iloc[i] = 7
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 20:
        df_mean['wind'].iloc[i] = 8
    elif df_mean['VELMEDIA'].iloc[i]*3.6 > 10:
        df_mean['wind'].iloc[i] = 9
    elif df_mean['VELMEDIA'].iloc[i]*3.6 < 10:
        df_mean['wind'].iloc[i] = 10

C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [54]:
df_mean.head()

,ALTITUD,TMEDIA,PRECIPITACION,TMIN,TMAX,DIR,VELMEDIA,RACHA,SOL,PRESMAX,PRESMIN,tmax,rain,wind
week,,,,,,,,,,,,,,
1,32,11.708571,0.202857,8.011429,15.417143,28.457143,2.780000,14.708571,NaN,NaN,NaN,6,9,9
2,32,10.165714,0.040000,6.728571,13.602857,30.885714,3.382857,16.685714,NaN,NaN,NaN,5,9,9
3,32,10.502857,2.554286,6.940000,14.082857,30.285714,2.831429,14.020000,NaN,NaN,NaN,5,9,9
4,32,11.206667,2.193103,8.053333,14.326667,33.300000,3.110000,15.663333,NaN,NaN,NaN,5,9,9
5,32,10.964286,1.292857,7.253571,14.664286,28.821429,2.600000,14.267857,NaN,NaN,NaN,5,9,10


## Assigning a CWI score to each week

Having all relative scores for any given week (max temperature, rain and wind speed) we can proceed to assign a score.

In [55]:
#Creating a new column by multiplying our individual scores by the corresponding %.

df_mean['CWI'] = df_mean['tmax']*0.4 + df_mean['rain']*0.4 + df_mean['wind']*0.2

In [56]:
#Our final dataframe needs only the last column (score).

df_score = pd.DataFrame(df_mean['CWI'])

In [57]:
df_score.head()

,CWI
week,
1,7.8
2,7.4
3,7.4
4,7.4
5,7.6


In [58]:
#Transposing the dataframe to condense all information in a single row.

df_t = df_score.T

In [59]:
#Reseting the index.

df_t.reset_index(drop=True, inplace=True)

In [60]:
#Finally, creating new columns for the weather station name and coordinates.

df_t['name'] = None
df_t['coords'] = None

In [61]:
df_t.head()

week,1,2,3,4,5,6,7,8,9,10,...,46,47,48,49,50,51,52,53,name,coords
0,7.8,7.4,7.4,7.4,7.6,7.8,8.0,8.0,8.0,8.4,...,7.8,8.0,7.8,8.0,7.8,8.0,8.0,8.0,None,None


In [62]:
c = df_t.columns.tolist()
r = df_t.iloc[0].tolist()

In [63]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 1 to 53
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CWI     53 non-null     object
dtypes: object(1)
memory usage: 2.9+ KB


# Applying the scoring system to every weather station

Now that we have all the necessary steps to transform a dataframe into our desired structure it's time to create a function (or a loop) that can automatically perform this operation on all *csv* containing meteo data. To access each folder we will be using **Path**.

This will be done in three steps, with three separate functions/loops:

1. Iterate through every *csv* file in a folder.
2. Transform each *csv* file with the previous methods and return a dicionary.
3. Create a master dataframe using all generated dictionaries.

## Creating our parser

Let's begin by defining the parameters that our function must meet:

1. Perform all necessary transformations on a given *csv* file.
2. Return a dictionary where the first key is the station name, the second one the coords, and the next 53 correspond to a week with their respective values reflecting that week's CWI.

In [162]:
#Defining our function.

def score_creator(file):
    df = pd.read_csv(file, sep=';') #Opening the file.
    try:
        df['PRECIPITACION'] = df.PRECIPITACION.str.replace(r"[a-zA-Z]",'')
    except: 
        pass
    df["PRECIPITACION"] = pd.to_numeric(df["PRECIPITACION"]) #Typecasting column.
    df["FECHA"] = df['FECHA'].apply(pd.to_datetime) #Converting from object to Datetime.
     
    df['TMAX'].fillna(df['TMAX'].mean(), inplace=True) #Replacing NaNs.
    df['PRECIPITACION'].fillna(df['PRECIPITACION'].mean(), inplace=True)
    df['VELMEDIA'].fillna(df['VELMEDIA'].mean(), inplace=True)
    
    
    filtered = df.loc[(df['FECHA'] >= '2016-01-01') & (df['FECHA'] < '2021-01-01')] #Grabbing data from the last 5 years.
    filtered.reset_index(drop=True, inplace=True) #Resetting index.
    
    filtered['week'] = None
    weekNumber = filtered['FECHA'].dt.week.tolist()
    filtered['week'] = weekNumber #Grouping by week.
    
    df_mean = filtered.groupby("week").mean() #Calculating the weekly average values.
    
    df_mean['tmax'] = None
    for i in range(len(df_mean)): #Assigning temp score.
        try:
            if df_mean['TMAX'].iloc[i] >= 39:
                df_mean['tmax'].iloc[i] = 0
            elif df_mean['TMAX'].iloc[i] > 38:
                df_mean['tmax'].iloc[i] = 1
            elif df_mean['TMAX'].iloc[i] > 37:
                df_mean['tmax'].iloc[i] = 2
            elif df_mean['TMAX'].iloc[i] > 36:
                df_mean['tmax'].iloc[i] = 3
            elif df_mean['TMAX'].iloc[i] > 35:
                df_mean['tmax'].iloc[i] = 4
            elif df_mean['TMAX'].iloc[i] > 33:
                df_mean['tmax'].iloc[i] = 5
            elif df_mean['TMAX'].iloc[i] > 31:
                df_mean['tmax'].iloc[i] = 6
            elif df_mean['TMAX'].iloc[i] > 29:
                df_mean['tmax'].iloc[i] = 7
            elif df_mean['TMAX'].iloc[i] > 27:
                df_mean['tmax'].iloc[i] = 8
            elif df_mean['TMAX'].iloc[i] > 26:
                df_mean['tmax'].iloc[i] = 9
            elif df_mean['TMAX'].iloc[i] > 23:
                df_mean['tmax'].iloc[i] = 10
            elif df_mean['TMAX'].iloc[i] > 20:
                df_mean['tmax'].iloc[i] = 9
            elif df_mean['TMAX'].iloc[i] > 19:
                df_mean['tmax'].iloc[i] = 8
            elif df_mean['TMAX'].iloc[i] > 18:
                df_mean['tmax'].iloc[i] = 7
            elif df_mean['TMAX'].iloc[i] > 15:
                df_mean['tmax'].iloc[i] = 6
            elif df_mean['TMAX'].iloc[i] > 11:
                df_mean['tmax'].iloc[i] = 5
            elif df_mean['TMAX'].iloc[i] > 7:
                df_mean['tmax'].iloc[i] = 4
            elif df_mean['TMAX'].iloc[i] > 0:
                df_mean['tmax'].iloc[i] > 3
            elif df_mean['TMAX'].iloc[i] <= 3:
                df_mean['tmax'].iloc[i] = 0
        except:
            df_mean['tmax'].iloc[i] = df_mean['tmax'].mean() #Filling missing values with the mean.
            
    df_mean['rain'] = None
    for i in range(len(df_mean)): #Assigning rain score.
        try:
            if df_mean['PRECIPITACION'].iloc[i] >= 40:
                df_mean['rain'].iloc[i] = -2
            elif df_mean['PRECIPITACION'].iloc[i] >= 25:
                df_mean['rain'].iloc[i] = -1
            elif df_mean['PRECIPITACION'].iloc[i] > 12:
                df_mean['rain'].iloc[i] = 0
            elif df_mean['PRECIPITACION'].iloc[i] > 10:
                df_mean['rain'].iloc[i] = 1
            elif df_mean['PRECIPITACION'].iloc[i] > 9:
                df_mean['rain'].iloc[i] = 2
            elif df_mean['PRECIPITACION'].iloc[i] > 8:
                df_mean['rain'].iloc[i] = 3
            elif df_mean['PRECIPITACION'].iloc[i] > 7:
                df_mean['rain'].iloc[i] = 4
            elif df_mean['PRECIPITACION'].iloc[i] > 6:
                df_mean['rain'].iloc[i] = 5
            elif df_mean['PRECIPITACION'].iloc[i] > 5:
                df_mean['rain'].iloc[i] = 6
            elif df_mean['PRECIPITACION'].iloc[i] > 4:
                df_mean['rain'].iloc[i] = 7
            elif df_mean['PRECIPITACION'].iloc[i] > 3:
                df_mean['rain'].iloc[i] = 8
            elif df_mean['PRECIPITACION'].iloc[i] > 0:
                df_mean['rain'].iloc[i] = 9
            elif df_mean['PRECIPITACION'].iloc[i] == 0.0:
                df_mean['rain'].iloc[i] = 10
        except:
            df_mean['rain'].iloc[i] = df_mean['rain'].mean() #Filling missing values with the mean.
            
    df_mean['wind'] = None
    for i in range(len(df_mean)): #Assigning wind score.
        try:
            if df_mean['VELMEDIA'].iloc[i]*3.6 > 70:
                df_mean['wind'].iloc[i] = -10
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 65:
                df_mean['wind'].iloc[i] = -8
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 60:
                df_mean['wind'].iloc[i] = -6
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 55:
                df_mean['wind'].iloc[i] = -4
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 50:
                df_mean['wind'].iloc[i] = -2
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 40:
                df_mean['wind'].iloc[i] = 0
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 35:
                df_mean['wind'].iloc[i] = 2
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 30:
                df_mean['wind'].iloc[i] = 6
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 25:
                df_mean['wind'].iloc[i] = 7
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 20:
                df_mean['wind'].iloc[i] = 8
            elif df_mean['VELMEDIA'].iloc[i]*3.6 > 10:
                df_mean['wind'].iloc[i] = 9
            elif df_mean['VELMEDIA'].iloc[i]*3.6 < 10:
                df_mean['wind'].iloc[i] = 10
        except:
            df_mean['wind'].iloc[i] = df_mean['wind'].mean() #Filling missing values with the mean.    
            
    df_mean['CWI'] = df_mean['tmax']*0.4 + df_mean['rain']*0.4 + df_mean['wind']*0.2 #Calculating the score.
    
    df_score = pd.DataFrame(df_mean['CWI']) #Keeping just the score.
    
    df_t = df_score.T #Transposing the dataframe.
    
    df_t['name'] = None
    df_t['coords'] = None
    
    df_t.reset_index(drop=True, inplace=True) #Resetting index.
    
    newname = (file.name)[:6].replace('-', '') #Assigning the weather station name.
    
    c = df_t.columns.tolist() #Creating a list of columns.
    r = df_t.iloc[0].tolist() #Creating a list of row values.
    
    keys = c #Creating a dictionary from our two lists.
    values = r
    dictionary = dict(zip(keys, values))
    
    dictionary['name'] = newname
    
    return dictionary #Returning our dictionary.

In [142]:
output = score_creator('B248-19710401-20210602.csv')

<ipython-input-141-395e5c47aac5>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PRECIPITACION'] = df.PRECIPITACION.str.replace(r"[a-zA-Z]",'')
<ipython-input-141-395e5c47aac5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['week'] = None
<ipython-input-141-395e5c47aac5>:22: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  weekNumber = filtered['FECHA'].dt.week.tolist()
<ipython-input-141-395e5c47aac5>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [143]:
#Testing our function.

output

{1: 6.6000000000000005,
 2: 6.6000000000000005,
 3: 4.800000000000001,
 4: 5.4,
 5: 6.6000000000000005,
 6: 6.200000000000001,
 7: 7.0,
 8: 7.0,
 9: 6.800000000000001,
 10: 6.6000000000000005,
 11: 7.3999999999999995,
 12: 5.2,
 13: 7.199999999999999,
 14: 7.199999999999999,
 15: 7.0,
 16: 7.3999999999999995,
 17: 6.6000000000000005,
 18: 7.3999999999999995,
 19: 7.8,
 20: 7.8,
 21: 9.0,
 22: 8.600000000000001,
 23: 9.0,
 24: 9.0,
 25: 9.4,
 26: 9.4,
 27: 8.600000000000001,
 28: 9.0,
 29: 9.4,
 30: 8.600000000000001,
 31: 9.0,
 32: 9.4,
 33: 9.0,
 34: 9.0,
 35: 9.4,
 36: 8.600000000000001,
 37: 7.6,
 38: 9.0,
 39: 9.0,
 40: 8.200000000000001,
 41: 7.8,
 42: 6.6000000000000005,
 43: 5.8,
 44: 7.3999999999999995,
 45: 5.800000000000001,
 46: 6.800000000000001,
 47: 6.4,
 48: 6.800000000000001,
 49: 6.0,
 50: 5.4,
 51: 4.0,
 52: 6.800000000000001,
 53: 6.6000000000000005,
 'name': 'B248',
 'coords': None}

## Creating the component that iterates through all csv files and running it

In [167]:
#This loop will iterate through all files in our designated folder and apply the yet to be defined function to them.

start = time.time() #Starting a timer.

dict_list = [] #We will be appending our output dictionaries here.

directory = 'csv' #The folder containing our csv files.
 
files = Path(directory).glob('*') #Grabbing all files.
for file in files:
    dict_list.append(score_creator(file)) #Applying our score function.
    
stop = time.time() #Stopping our timer.
duration = (stop - start) / 60
print('Minutes:', duration) #Returning the elapsed minutes.

test = pd.DataFrame(dict_list) #Creating the dataframe from the list of dictionaries.

<ipython-input-162-4be671f46f02>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PRECIPITACION'] = df.PRECIPITACION.str.replace(r"[a-zA-Z]",'')
<ipython-input-162-4be671f46f02>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['week'] = None
<ipython-input-162-4be671f46f02>:21: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  weekNumber = filtered['FECHA'].dt.week.tolist()
<ipython-input-162-4be671f46f02>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

Minutes: 4.688370108604431


We can see that many rows (weather stations) are missing or have missing weeks, that's because they have faulty data or the recording simply doesn't fill the last 5 years. The rows with missing data can be safely dropped.

In [191]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 55 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       264 non-null    float64
 1   2       264 non-null    float64
 2   3       264 non-null    float64
 3   4       263 non-null    float64
 4   5       262 non-null    float64
 5   6       262 non-null    float64
 6   7       263 non-null    float64
 7   8       263 non-null    float64
 8   9       263 non-null    float64
 9   10      263 non-null    float64
 10  11      265 non-null    float64
 11  12      265 non-null    float64
 12  13      265 non-null    float64
 13  14      265 non-null    float64
 14  15      265 non-null    float64
 15  16      265 non-null    float64
 16  17      263 non-null    float64
 17  18      264 non-null    float64
 18  19      263 non-null    float64
 19  20      263 non-null    float64
 20  21      262 non-null    float64
 21  22      263 non-null    float64
 22  23

In [200]:
#Dropping the rows with missing data.

meteo = test.dropna(thresh=52)

In [203]:
#Finally, saving our resulting dataframe.

meteo.to_csv('meteo.csv', index=False)